## Requirements

In [1]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
import keras
from keras import layers
import matplotlib.pyplot as plt
from matplotlib import colors
import h5py as h5
import numpy as np

2022-03-08 09:04:17.409363: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-08 09:04:17.409394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Getting data form files

In [2]:
with h5.File('./train_catvnoncat.h5', 'r') as f:
    train_set_x = list(f.keys())[1]
    train_set_y = list(f.keys())[2]
    
    cat_data = list(f[train_set_y])
    image_data = list(f[train_set_x])

with h5.File('./test_catvnoncat.h5', 'r') as f:
    test_set_x = list(f.keys())[1]
    test_set_y = list(f.keys())[2]
    
    cat_data_test = list(f[test_set_y])
    image_data_test = list(f[test_set_x])

## Normalize Images Function

We use this function to convert the images to gray_scale and return them as a single vector (using reshape).

In [3]:
def normalize_images(images):
    gray_images = []
    for image in images:
        gray_image = []
        for line in image:
            new_line = []
            for pixel in line:
                gray_value = (0.2126 * pixel[0] + 0.7152 * pixel[1] + 0.0722 * pixel[2]) / 255
                new_line.append(gray_value)
            gray_image.append(new_line)
        gray_images.append(gray_image)
    
    gray_images = np.array(gray_images)
    n_samples = len(gray_images)
    gray_images = gray_images.reshape((n_samples, -1))
    return gray_images

## Using Perceptron

In [4]:
normalized_images = normalize_images(image_data)

perceptron = Perceptron(random_state=1)
perceptron.fit(normalized_images, cat_data)

normalized_test_images = normalize_images(image_data_test)
y_pred = perceptron.predict(normalized_test_images)

## Printing Perceptron Results

In [5]:
print(confusion_matrix(cat_data_test, y_pred))
print(confusion_matrix(cat_data_test, y_pred,normalize='true'))
print('Accuracy: %.2f' % accuracy_score(cat_data_test, y_pred))

[[10  7]
 [10 23]]
[[0.58823529 0.41176471]
 [0.3030303  0.6969697 ]]
Accuracy: 0.66


## Using MLPClassifier

In [6]:
mlp = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=300, random_state=8)
mlp.fit(normalized_images, cat_data)
mlp_predict = mlp.predict(normalized_test_images)

## Printing MLPClassifier Results

In [7]:
print(confusion_matrix(cat_data_test, mlp_predict))
print(confusion_matrix(cat_data_test, mlp_predict,normalize='true'))
print(classification_report(cat_data_test, mlp_predict))

[[ 5 12]
 [ 2 31]]
[[0.29411765 0.70588235]
 [0.06060606 0.93939394]]
              precision    recall  f1-score   support

           0       0.71      0.29      0.42        17
           1       0.72      0.94      0.82        33

    accuracy                           0.72        50
   macro avg       0.72      0.62      0.62        50
weighted avg       0.72      0.72      0.68        50



Using Convolutional Network

In [4]:
num_classes = 10
input_shape = (28,28,1)

train_images = cat_data
train_labels = image_data
test_images = cat_data_test
test_labels = image_data_test

train_images = normalize_images(image_data)
test_images = normalize_images(image_data_test)

train_images = np.expand_dims(train_images,-1)
test_images = np.expand_dims(test_images,-1)

print("train images shape:", train_images.shape)
print(train_images.shape[0], "train samples")
print(test_images.shape[0], "test samples")


train images shape: (209, 4096, 1)
209 train samples
50 test samples


In [5]:
model_cnn = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

2022-03-08 09:04:40.968215: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-08 09:04:40.968277: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-08 09:04:40.968317: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ibm4381): /proc/driver/nvidia/version does not exist
